<a href="https://colab.research.google.com/github/gitmystuff/DTSC5810/blob/main/Week_03-Data_Organization/SQL_II.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SQL II

* https://www.w3schools.com/sql/default.asp
* https://www.sqlitetutorial.net/sqlite-sample-database/
* https://www.codeproject.com/Articles/878359/Data-Modelling-using-ERD-with-Crow-Foot-Notation

Northwind

* https://github.com/jpwhite3/northwind-SQLite3
* https://en.wikiversity.org/wiki/Database_Examples/Northwind

In [1]:
import sqlite3

connection = sqlite3.connect('/content/drive/MyDrive/DTSC5810/Week 03 - Data Organization/northwind.db')
cursor = connection.cursor()

In [2]:
query = "SELECT * FROM sqlite_master where type='table';"
cursor.execute(query)
cursor.fetchall()

[('table',
  'Categories',
  'Categories',
  2,
  'CREATE TABLE [Categories]\n(      [CategoryID] INTEGER PRIMARY KEY AUTOINCREMENT,\n       [CategoryName] TEXT,\n       [Description] TEXT,\n       [Picture] BLOB\n)'),
 ('table',
  'sqlite_sequence',
  'sqlite_sequence',
  3,
  'CREATE TABLE sqlite_sequence(name,seq)'),
 ('table',
  'CustomerCustomerDemo',
  'CustomerCustomerDemo',
  28,
  'CREATE TABLE [CustomerCustomerDemo](\n   [CustomerID]TEXT NOT NULL,\n   [CustomerTypeID]TEXT NOT NULL,\n   PRIMARY KEY ("CustomerID","CustomerTypeID"),\n   FOREIGN KEY ([CustomerID]) REFERENCES [Customers] ([CustomerID]) \n\t\tON DELETE NO ACTION ON UPDATE NO ACTION,\n\tFOREIGN KEY ([CustomerTypeID]) REFERENCES [CustomerDemographics] ([CustomerTypeID]) \n\t\tON DELETE NO ACTION ON UPDATE NO ACTION\n)'),
 ('table',
  'CustomerDemographics',
  'CustomerDemographics',
  30,
  'CREATE TABLE [CustomerDemographics](\n   [CustomerTypeID]TEXT NOT NULL,\n   [CustomerDesc]TEXT,\n    PRIMARY KEY ("CustomerType

In [3]:
query = '''
SELECT sql
FROM sqlite_schema
WHERE name = 'Products';
'''
cursor.execute(query)
cursor.fetchall()

[("CREATE TABLE [Products](\n   [ProductID]INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,\n   [ProductName]TEXT NOT NULL,\n   [SupplierID]INTEGER,\n   [CategoryID]INTEGER,\n   [QuantityPerUnit]TEXT,\n   [UnitPrice]NUMERIC DEFAULT 0,\n   [UnitsInStock]INTEGER DEFAULT 0,\n   [UnitsOnOrder]INTEGER DEFAULT 0,\n   [ReorderLevel]INTEGER DEFAULT 0,\n   [Discontinued]TEXT NOT NULL DEFAULT '0',\n    CHECK ([UnitPrice]>=(0)),\n    CHECK ([ReorderLevel]>=(0)),\n    CHECK ([UnitsInStock]>=(0)),\n    CHECK ([UnitsOnOrder]>=(0)),\n\tFOREIGN KEY ([CategoryID]) REFERENCES [Categories] ([CategoryID]) \n\t\tON DELETE NO ACTION ON UPDATE NO ACTION,\n\tFOREIGN KEY ([SupplierID]) REFERENCES [Suppliers] ([SupplierID]) \n\t\tON DELETE NO ACTION ON UPDATE NO ACTION\n)",)]

In [ ]:
query = '''
SELECT ProductName from Products LIMIT 10;
'''
cursor.execute(query)
cursor.fetchall()

[('Chai',),
 ('Chang',),
 ('Aniseed Syrup',),
 ("Chef Anton's Cajun Seasoning",),
 ("Chef Anton's Gumbo Mix",),
 ("Grandma's Boysenberry Spread",),
 ("Uncle Bob's Organic Dried Pears",),
 ('Northwoods Cranberry Sauce',),
 ('Mishi Kobe Niku',),
 ('Ikura',)]

In [ ]:
query = '''
SELECT * FROM Products LIMIT 5;
'''
cursor.execute(query)
cursor.fetchall()

[(1, 'Chai', 1, 1, '10 boxes x 20 bags', 18, 39, 0, 10, '0'),
 (2, 'Chang', 1, 1, '24 - 12 oz bottles', 19, 17, 40, 25, '0'),
 (3, 'Aniseed Syrup', 1, 2, '12 - 550 ml bottles', 10, 13, 70, 25, '0'),
 (4,
  "Chef Anton's Cajun Seasoning",
  2,
  2,
  '48 - 6 oz jars',
  22,
  53,
  0,
  0,
  '0'),
 (5, "Chef Anton's Gumbo Mix", 2, 2, '36 boxes', 21.35, 0, 0, 0, '1')]

In [ ]:
query = '''
SELECT * FROM Products;
'''
cursor.execute(query)
result = cursor.fetchall()

In [ ]:
# create dataframe
import pandas as pd

df = pd.DataFrame(result)
df.columns = [
    'ProductID', 'ProductName', 'SupplierID',
    'CategoryID', 'QuantityPerUnit', 'UnitPrice',
    'UnitsInStock', 'UnitsOnOrder', 'ReorderLevel',
    'Discontinued'
]
df.head()

,ProductID,ProductName,SupplierID,CategoryID,QuantityPerUnit,UnitPrice,UnitsInStock,UnitsOnOrder,ReorderLevel,Discontinued
0,1,Chai,1,1,10 boxes x 20 bags,18.00,39,0,10,0
1,2,Chang,1,1,24 - 12 oz bottles,19.00,17,40,25,0
2,3,Aniseed Syrup,1,2,12 - 550 ml bottles,10.00,13,70,25,0
3,4,Chef Anton's Cajun Seasoning,2,2,48 - 6 oz jars,22.00,53,0,0,0
4,5,Chef Anton's Gumbo Mix,2,2,36 boxes,21.35,0,0,0,1


In [ ]:
query = '''
SELECT p.ProductName, c.CategoryName, p.UnitsInStock, p.UnitsOnOrder
FROM Products as p, Categories as c
WHERE p.CategoryID = c.CategoryID;
'''
cursor.execute(query)
result = cursor.fetchall()

In [ ]:
# create dataframe
import pandas as pd

df = pd.DataFrame(result)
df.columns = [
    'ProductName', 'CategoryName', 'UnitsInStock', 'UnitsOnOrder'
]
df.head()

,ProductName,CategoryName,UnitsInStock,UnitsOnOrder
0,Chai,Beverages,39,0
1,Chang,Beverages,17,40
2,Aniseed Syrup,Condiments,13,70
3,Chef Anton's Cajun Seasoning,Condiments,53,0
4,Chef Anton's Gumbo Mix,Condiments,0,0


## Aggregates

* Min, Max, Count, Sum, Avg

In [7]:
import pandas as pd

query = '''
SELECT c.CategoryName, AVG(p.UnitsInStock), AVG(p.UnitsOnOrder)
FROM Products as p, Categories as c
WHERE p.CategoryID = c.CategoryID
GROUP BY c.CategoryName;
'''
cursor.execute(query)

df = pd.DataFrame(cursor.fetchall())
df.columns = [
    'CategoryName', 'UnitsInStock Avg', 'UnitsOnOrder Avg'
]
df

,CategoryName,UnitsInStock Avg,UnitsOnOrder Avg
0,Beverages,46.583333,5.000000
1,Condiments,42.250000,14.166667
2,Confections,29.692308,13.846154
3,Dairy Products,39.300000,14.000000
4,Grains/Cereals,44.000000,12.857143
5,Meat/Poultry,27.500000,0.000000
6,Produce,20.000000,4.000000
7,Seafood,58.416667,10.000000


## Common Table Expression (CTE)

* https://learnsql.com/blog/what-is-common-table-expression/

## SQL to CSV

In [8]:
import pandas as pd

query = '''
SELECT p.ProductName, c.CategoryName, p.UnitPrice, p.UnitsInStock, p.UnitsOnOrder, p.Discontinued
FROM Products as p, Categories as c
WHERE p.CategoryID = c.CategoryID;
'''
cursor.execute(query)

df = pd.DataFrame(cursor.fetchall())
df.columns = [
    'ProductName', 'CategoryName', 'UnitPrice',
    'UnitsInStock', 'UnitsOnOrder', 'Discontinued'
]

df.to_csv('products.csv', index=False)

In [9]:
df = pd.read_csv('products.csv')
df.head()

,ProductName,CategoryName,UnitPrice,UnitsInStock,UnitsOnOrder,Discontinued
0,Chai,Beverages,18.00,39,0,0
1,Chang,Beverages,19.00,17,40,0
2,Aniseed Syrup,Condiments,10.00,13,70,0
3,Chef Anton's Cajun Seasoning,Condiments,22.00,53,0,0
4,Chef Anton's Gumbo Mix,Condiments,21.35,0,0,1
